# Notebook App for Data Enrichment Using LangGrpah and Tavily

## Introduction
Welcome to the Data Enrichment Sheet!

This sheet is designed to enhance your data by populating missing values using advanced search capabilities from Tavily and the LangGraph Framework. It accepts either a CSV or Excel file path or a Google Sheet name (follow the instructions in the "Google Colab Set Up" section).

For Google Sheets and Excel files, the agent will create an additional sheet containing the populated data table. For CSV files, a new CSV file will be generated with the enriched data provided by the agent.

For detailed instructions on how to use this sheet and call the agent to start enriching your data, please refer to the "Call Agent" section below.

## Requirements


In [ ]:
!pip install langgraph tavily-python openai openpyxl

## Libraries

In [ ]:
import os
import pandas as pd
import numpy as np
import json
import asyncio
from openpyxl import load_workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from typing import TypedDict, Annotated
from openai import AsyncOpenAI
from tavily import AsyncTavilyClient
from langgraph.graph import StateGraph, END

## Google Colab Set Up

To connent with Google Sheets upload this notebook to Google Colab. Uncomment the following lines and ensure to input your Google Sheet name for your agent in the Call Agent section.

In [ ]:
# import gspread # Python API for Google Sheets
# from google.colab import auth
# from google.auth import default

# # Authenticate and create the gspread client
# auth.authenticate_user()
# creds, _ = default()

# gc = gspread.authorize(creds)

## Build Master Agent

### Master's Agent State

In [ ]:
class AgentState(TypedDict):
  raw_data: pd.DataFrame                  # original panda data frame
  new_data: pd.DataFrame                  # populated panda data frame
  colab: str                              # google sheet name
  excel: str                              # path to Excel file
  csv: str                                # path to CSV file

### Master's Agent class

In [ ]:
class DataEnrichmentAgent():
  def __init__(self):
    # Initialize agents
    data_agent = DataAgent()
    enrich_agent = EnrichAgent()

    workflow = StateGraph(AgentState)

    workflow.add_node("get_data", data_agent.get_df)
    workflow.add_node("enrich_data", enrich_agent.run)
    workflow.add_node("write_data", data_agent.write_df)

    workflow.set_entry_point("get_data")

    workflow.add_edge("get_data", "enrich_data")
    workflow.add_edge("enrich_data", "write_data")
    workflow.add_edge("write_data", END)

    self.workflow = workflow.compile()

## Sub Agents

### Data Agent

In [ ]:
class DataAgent():
  def __init__(self):
    pass

  # Google Sheets Integration through Colab
  def connect_to_google_sheets_colab(self, sheet_name):
    sheet = gc.open(sheet_name).sheet1
    data = sheet.get_all_records()
    df = pd.DataFrame(data)
    df.replace('', np.nan, inplace=True)
    return df

  def get_df(self, state: AgentState):
    # Function to get Data Frame and store in state
    if state['colab']:
      df = self.connect_to_google_sheets_colab(state['colab'])
      return {"raw_data": df}
    elif state['csv']:
      df = pd.read_csv(state['csv'])
      return {"raw_data": df}
    elif state['excel']:
      df = pd.read_excel(state['excel'])
      return {"raw_data": df}
    else:
        print("Either colab or csv or excel argument is required.")
        return

  def write_df(self, state: AgentState):
    df = state['new_data']
    if state['colab']:
        # Use existing Colab authentication (assuming `gc` is a gspread client)
        spreadsheet = gc.open(state['colab'])

        # Add a new sheet
        sheet_name = f"AgentSheet{len(spreadsheet.worksheets()) + 1}"
        spreadsheet.add_worksheet(title=sheet_name, rows=str(df.shape[0]), cols=str(df.shape[1]))
        new_sheet = spreadsheet.worksheet(sheet_name)

        # Write DataFrame to the new sheet
        new_sheet.update([df.columns.values.tolist()] + df.values.tolist())
        print(f"Data written to new sheet '{sheet_name}' in Google Sheets document '{state['colab']}'.")

    elif state['excel']:
      # Load the existing workbook
      wb = load_workbook(state['excel'])

      # Generate a new sheet name based on existing sheets count
      sheet_name = f"AgentSheet{len(wb.sheetnames) + 1}"

      # Create a new sheet and write data
      ws = wb.create_sheet(title=sheet_name)

      # Write DataFrame to the new sheet
      for r in dataframe_to_rows(df, index=False, header=True):
        ws.append(r)

      # Save changes to the Excel file
      wb.save(state['excel'])
      wb.close()
      print(f"Data written to new sheet '{sheet_name}' in Excel document '{state['excel']}'.")

    elif state['csv']:
      # Extract file name from the csv path and construct new name
      file_name = os.path.basename(state['csv'])
      # Output DataFrame to a CSV file
      output_csv = f"AgentSheet{file_name}" # construct new file path
      df.to_csv(output_csv, index=False)
      print(f"Data written to CSV file '{output_csv}'.")

### Enrich Agent

In [ ]:
# Colors
RED = '\033[91m'
GREEN = '\033[92m'
BLUE = '\033[94m'
YELLOW = '\033[93m'
ENDC = '\033[0m'

In [ ]:
class EnrichAgent():
  def __init__(self):
    # Maximum number of columns to try to resolve at once
    self.MAX_COLS_PER_PASS = 5

    # Maximum number of fill-in passes to attempt
    self.MAX_PASSES = 5

  async def generate_search_query(self, head, columns):
    """
    This function takes the head of the table (all column names) and the names of the columns that it needs to fill in, and generates a search prompt for Tavily
    """
    prompt = f"""You are a researcher with the task of filling in a spreadsheet. The spreadsheet contains the columns {str(head)} and {str(columns)} have not been filled in yet.
    Write a web search query for a search engine that you will use to fill in each row of the spreadsheet one by one. In the query, replace the name of the entry you are researching with $ENTRY
    Respond only with the query.
    """

    response = await openai.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    return response.choices[0].message.content.strip('\'"')

  async def fill_in_row(self, df, head, row_index, columns, search_query):
    """
    This function takes a data frame, a row to complete, a list of column names to complete and the query to search.
    It calls the Tavily API to retrieve information using the search_query and prompts OpenAI to extract column values from the response
    """
    # if the search_query is a coroutine, await it
    if asyncio.iscoroutine(search_query):
        search_query = await search_query

    entry = df.iloc[row_index][head[0]]

    search_query = search_query.replace("$ENTRY", entry)
    print(f"{BLUE}Tavily Search: {search_query}{ENDC}")
    tavily_response = await tavily.search(search_query)

    # Only save the title and content from each Tavily result
    research_results = [{'title': r['title'], 'content': r['content']} for r in tavily_response['results']]

    # To ensure consistency with the existing data, come up with an example based on existing data in the df
    example = {}
    for col in columns:
      for i in range(len(df)):
        if i != row_index and not pd.isna(df.iloc[i][col]):
          example[col] = df.iloc[i][col]
          break

    example_str = f'Example: {str(example)}\n' if len(example) > 0 else ''

    prompt = f"You are filling in a spreadsheet using online sources. The following fields need to be filled in for the entry '{entry}': {', '.join(columns)}\n" \
              "Using the data below, find values for as many of these fields as you can. Reply with a JSON object, linking each field to its value.\n" \
            f"{example_str}" \
              "Values must be written in a spreadsheet friendly format. If the data is not sufficient to complete a field, simply omit it from the result object.\n" \
              "Do not incude any text other than the JSON object in your response.\n\n" \
                      f"{str(research_results)}"

    response = await openai.chat.completions.create(
        model="gpt-4o",
        response_format={ "type": "json_object" },
        messages=[
            {"role": "user", "content": prompt}
        ]
    )

    json_str = response.choices[0].message.content
    try:
      fields = json.loads(json_str)
    except json.JSONDecodeError as e:
      print(f"{RED}Failed to parse JSON: {json_str}{ENDC}")
      return

    for field, value in fields.items():
      if field not in columns:
        print(f"{RED}LLM returned an invalid field '{field}'{ENDC}")
        continue

      col_index = df.columns.get_loc(field)
      if not pd.isna(df.iloc[row_index][field]):
        print(f"{YELLOW}Skipping field '{field}' as it already has a value:{ENDC}", df.iloc[row_index][field])
        continue

      df.at[row_index, field] = str(value)
      print(f"{GREEN}Filled in field '{field}' with value '{value}'{ENDC}")

  async def run(self, state: AgentState):
    """
    Main Function:
    First, we try to fill in the maximum number of column in one pass for each row. We then loop and try to fill in the missing columns
    """
    df = state['raw_data'].copy()
    head = list(df.columns)
    # First pass: try to fill in as much as possible
    general_query = await self.generate_search_query(head, head[1:self.MAX_COLS_PER_PASS+1])

    coroutines = []
    for row_index in range(len(df)):
      has_missing_columns = False
      for col_index in range(len(head)):
        if pd.isna(df.iloc[row_index][head[col_index]]):
          has_missing_columns = True
          break


      if not has_missing_columns:
        print(f"{YELLOW}No missing columns for row {row_index}{ENDC}")
        continue

      coroutines.append(self.fill_in_row(df, head, row_index, head[1:], general_query))

    await asyncio.gather(*coroutines)

    # Now, fill in the missing fields for each row
    for _ in range(self.MAX_PASSES):
      all_completed = True

      coroutines = []
      for row_index in range(len(df)):
        missing_columns = []
        for col_index in range(len(head)):
          if pd.isna(df.iloc[row_index][head[col_index]]) and len(missing_columns) < self.MAX_COLS_PER_PASS:
            missing_columns.append(head[col_index])

        if len(missing_columns) > 0:
          all_completed = False
          print(f"Missing columns for row {row_index}: {', '.join(missing_columns)}")
          query = self.generate_search_query(head, missing_columns)
          coroutines.append(self.fill_in_row(df, head, row_index, missing_columns, query))
        else:
          print(f"No missing columns for row {row_index}")

      if all_completed:
        break

      await asyncio.gather(*coroutines)

    # Check if everything was completed
    all_completed = True
    for row_index in range(len(df)):
      for col_index in range(len(head)):
        if pd.isna(df.iloc[row_index][head[col_index]]):
          all_completed = False
          break

    if all_completed:
      print(f"{GREEN}All rows completed{ENDC}")
    else:
      print(f"{RED}Not all rows completed{ENDC}")
    return {"new_data": df}


# Call Agent

In [ ]:
# Set Your API Keys
OPENAI_API_KEY = "YOUR OPENAI API KEY"
TAVILY_API_KEY = "YOUR TAIVLY API KEY"

openai = AsyncOpenAI(api_key=OPENAI_API_KEY)
tavily = AsyncTavilyClient(TAVILY_API_KEY)

In [ ]:
# Define the input data for your agent, specifying the path to either a CSV file or an Excel file,
# and optionally the name of your Google Sheet
csv_path = "your_csv_path"       #E.g "Desktop/my_csv.scv"
excel_path = "test.xlsx"         #E.g "Desktop/my_exl.xlsx"
colab = "google_sheet_name"      #E.g "My_Google_Sheet"

In [ ]:
my_agent = DataEnrichmentAgent()
input = {
    # Example: Uncomment and specify the path to your CSV file above
    # "csv": csv_path,

    # Example: Uncomment and specify the path to your Excel file above
    "excel": excel_path,

    # Example: Uncomment and specify your Google Sheet name above
    # "colab": colab
}
result = await my_agent.workflow.ainvoke(input)

#### Your original Data

In [ ]:
result['raw_data']

#### Enriched Data

In [ ]:
result['new_data']